In [4]:
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from NN_files.gbfeatures import GradientBoostingFeatureGenerator
from sklearn.externals import joblib
import random
import os

# os.path.isfile(fname)

random.seed(0)


class ReadDataset(Dataset):
    """CERN kaagle dataset."""

    def __init__(self, csv_file, for_test=False):
        """
        Args:
            csv_file (str): Path to the csv file with the students data.

        """
        self.df = pd.read_csv(csv_file).drop(columns="BUTTER")
        self.df.columns = self.df.columns.str.replace(" ", "")

        try:
            self.df = self.df.drop(columns=["Unnamed:0"])
        except Exception:
            pass

        # Target
        self.target = "signal"
        # If we read test, not load the target
        if for_test == False:
            # Save target and predictors
            self.X = self.df.drop(self.target, axis=1)
            self.y = self.df[self.target]

        else:
            self.X = self.df

        # Create features
        self.X = self.transform(self.X)
        self.X = self.feature_engineering(self.X)


        # If the scaler does not exist create it
        if os.path.isfile("output/scaler.save") == False:
            self.scaler = MinMaxScaler()
            self.scaler = self.scaler.fit(self.X)
            ## Save scaler to be later used on the prediction
            joblib.dump(self.scaler, "output/scaler.save")
        # If the scale exist, load it
        else:
            self.scaler = joblib.load("output/scaler.save")
        ## Scale data
        self.X = pd.DataFrame(self.scaler.transform(self.X), columns=self.X.columns)
        print('---')
        print(self.X.shape)
        ## GB features
        sample_size = int(self.X.shape[0]/10)
        if os.path.isfile("output/gbFreat.save") == False:
            self.gb_feat = GradientBoostingFeatureGenerator()
            self.gb_feat.fit(self.X.head(sample_size),self.y.head(sample_size))
            joblib.dump(self.scaler, "output/gbFeat.save")
        else:
            self.gb_feat = joblib.load("output/gbFeat.save")
        self.X = pd.DataFrame(self.gb_feat.transform(self.X))

        print(self.X.shape)


    def transform(self, df):
        # Did not work
        df["Kst_892_0_cosThetaH_arc"] = np.arccos(df["Kst_892_0_cosThetaH"])
        df["Kst_892_0_cosThetaH_arc_sin"] = np.sin(df.Kst_892_0_cosThetaH_arc)

        df["Kplu_pXKst_892costheta"] = df["Kplus_P"] * df["Kst_892_0_cosThetaH"]

        df["momentum_by_shortest_dist"] = df["Kplus_P"] / df["Kplus_IP_OWNPV"]

        df["momentum_sum"] = df["B_PT"] + df["Kplus_P"] + df["gamma_PT"]

        # Worked individually
        df["Kplu_p_div_Kst_892costheta"] = df["Kplus_P"] / df["Kst_892_0_cosThetaH"]

        # ETA (estimated time of arrival) inversions -- worked
        df["Kplus_ETA_inv"] = 1 / df["Kplus_ETA"]
        df["piminus_ETA_inv"] = 1 / df["piminus_ETA"]

        # Momentum

        df["total_mom_2"] = df["B_PT"] ** 2 + df["gamma_PT"] ** 2 + df["Kplus_P"] ** 2
        df["total_mom_sum"] = df["B_PT"] + df["gamma_PT"] + df["Kplus_P"]

        # All mom ratios worked together
        df["mom_rat_1"] = df["B_PT"] / df["Kplus_P"]
        df["mom_rat_2"] = df["B_PT"] / df["gamma_PT"]

        df["mom_rat_3"] = df["gamma_PT"] / df["B_PT"]
        df["mom_rat_4"] = df["gamma_PT"] / df["Kplus_P"]

        df["mom_rat_5"] = df["Kplus_P"] / df["B_PT"]
        df["mom_rat_6"] = df["Kplus_P"] / df["gamma_PT"]

        df["ThetaH"] = np.arccos(df["Kst_892_0_cosThetaH"])

        df["Kplus_P_x"] = df["Kplus_P"] * np.sin(df["ThetaH"])
        df["Kplus_P_y"] = df["Kplus_P"] * np.cos(df["ThetaH"])

        df["mom_consev1"] = df["gamma_PT"] ** 2 + df["Kplus_P"] ** 2 - df["B_PT"] ** 2
        df["mom_consev1_1"] = ((df["gamma_PT"] + df["Kplus_P"])) ** 2 - df["B_PT"] ** 2

        # B meson ratios
        df["mesB_ratio_1"] = df["B_FDCHI2_OWNPV"] / df["B_IPCHI2_OWNPV"]
        df["mesB_ratio_2"] = df["B_FDCHI2_OWNPV"] / df["B_PT"]

        df["mesB_ratio_3"] = df["B_IPCHI2_OWNPV"] / df["B_PT"]
        df["mesB_ratio_4"] = df["B_IPCHI2_OWNPV"] / df["B_PT"]

        df["mesB_ratio_5"] = df["B_PT"] / df["B_IPCHI2_OWNPV"]
        df["mesB_ratio_6"] = df["B_PT"] / df["B_FDCHI2_OWNPV"]

        # Neither Improved neither worst
        df["kst_thetaH"] = np.arccos(df.Kst_892_0_cosThetaH)  # * 180 / np.pi
        df["kst_thetaH_sin"] = np.sin(df["kst_thetaH"])
        df["kst_thetaH_cos"] = np.cos(df["kst_thetaH"])
        df["kst_thetaH_sin_cos"] = np.sin(df["kst_thetaH"]) - np.cos(df["kst_thetaH"])
        df["kst_thetaH_tan"] = np.tan(df["kst_thetaH"])
        df["kst_thetaH_exp"] = np.exp(df["kst_thetaH"])
        df["kst_thetaH_exp_1"] = np.exp(-df["kst_thetaH"])

        df["B_DIRA_OWNPV_angle"] = np.arccos(df["B_DIRA_OWNPV"])  # * 180 / np.pi
        df["B_DIRA_OWNPV__cos"] = np.cos(df["B_DIRA_OWNPV_angle"])
        df["B_DIRA_OWNPV__sin"] = np.sin(df["B_DIRA_OWNPV_angle"])
        df["B_DIRA_OWNPV__tan"] = np.tan(df["B_DIRA_OWNPV_angle"])
        df["B_DIRA_OWNPV__sin_cos"] = np.sin(df["B_DIRA_OWNPV_angle"]) - np.cos(
            df["B_DIRA_OWNPV_angle"]
        )
        df["B_DIRA_OWNPV_exp"] = np.exp(df["B_DIRA_OWNPV_angle"])
        df["B_DIRA_OWNPV_exp_1"] = np.exp(-df["B_DIRA_OWNPV_angle"])

        # Momentum multiplication
        df["mom1_exp"] = df["mom_rat_1"] * df["B_DIRA_OWNPV_exp"]
        df["mom2_exp"] = df["mom_rat_2"] * df["B_DIRA_OWNPV_exp"]
        df["mom3_exp"] = df["mom_rat_3"] * df["B_DIRA_OWNPV_exp"]
        df["mom4_exp"] = df["mom_rat_4"] * df["B_DIRA_OWNPV_exp"]
        df["mom5_exp"] = df["mom_rat_5"] * df["B_DIRA_OWNPV_exp"]
        df["mom6_exp"] = df["mom_rat_6"] * df["B_DIRA_OWNPV_exp"]
        df["mom7_exp"] = df["B_PT"] * df["B_DIRA_OWNPV_exp"]
        df["mom8_exp"] = df["Kplus_P"] * df["B_DIRA_OWNPV_exp"]
        df["mom9_exp"] = df["gamma_PT"] * df["B_DIRA_OWNPV_exp"]

        df["mom11_exp"] = df["mom_rat_1"] * df["kst_thetaH"]
        df["mom22_exp"] = df["mom_rat_2"] * df["kst_thetaH"]
        df["mom33_exp"] = df["mom_rat_3"] * df["kst_thetaH"]
        df["mom44_exp"] = df["mom_rat_4"] * df["kst_thetaH"]
        df["mom55_exp"] = df["mom_rat_5"] * df["kst_thetaH"]
        df["mom66_exp"] = df["mom_rat_6"] * df["kst_thetaH"]
        df["mom77_exp"] = df["B_PT"] * df["kst_thetaH"]
        df["mom88_exp"] = df["Kplus_P"] * df["kst_thetaH"]
        df["mom99_exp"] = df["gamma_PT"] * df["kst_thetaH"]

        df["Kplus_ETA_2"] = df["Kplus_ETA"] ** 2
        df["Kplus_ETA_exp"] = np.exp(df["Kplus_ETA"])
        df["Kplus_ETA_theta"] = df["Kplus_ETA"] * df["kst_thetaH"]
        df["Kplus_ETA_div_theta"] = df["Kplus_ETA"] / df["kst_thetaH"]
        df["Kplus_ETA_theta_sin"] = df["Kplus_ETA"] * df["kst_thetaH_sin"]
        df["Kplus_ETA_theta_tan"] = df["Kplus_ETA"] * df["kst_thetaH_tan"]

        df["piminus_ETA_2"] = df["piminus_ETA"] ** 2
        df["piminus_ETA_exp"] = np.exp(df["piminus_ETA"])
        df["piminus_ETA_theta"] = df["piminus_ETA"] * df["kst_thetaH"]
        df["piminus_ETA_div_theta"] = df["piminus_ETA"] / df["kst_thetaH"]
        df["piminus_ETA_theta_sin"] = df["piminus_ETA"] * df["kst_thetaH_sin"]
        df["piminus_ETA_theta_tan"] = df["piminus_ETA"] * df["kst_thetaH_tan"]

        # Improves
        df["Kplus_by_eta"] = df["Kplus_P"] / df["Kplus_ETA"]

        df["Kplus_piminus_by_eta"] = df["Kplus_P"] / df["piminus_ETA"]

        df["ETA_add"] = df["Kplus_ETA"] + df["piminus_ETA"]
        df["ETA_rest"] = df["Kplus_ETA"] - df["piminus_ETA"]
        df["ETA_mult"] = df["Kplus_ETA"] * df["piminus_ETA"]
        df["ETA_inv"] = df["Kplus_ETA"] / df["piminus_ETA"]
        df["ETA_inv1"] = df["piminus_ETA"] / df["Kplus_ETA"]

        df["Kplus_ETA_inv"] = df["Kplus_ETA"]
        df["piminus_ETA_inv"] = df["piminus_ETA"]

        df["ETA_add_inv"] = df["Kplus_ETA_inv"] + df["piminus_ETA_inv"]
        df["ETA_rest_inv"] = df["Kplus_ETA_inv"] - df["piminus_ETA_inv"]
        df["ETA_mult_inv"] = df["Kplus_ETA_inv"] * df["piminus_ETA_inv"]
        df["ETA_inv_inv"] = df["Kplus_ETA_inv"] / df["piminus_ETA_inv"]
        df["ETA_inv1_inv"] = df["piminus_ETA_inv"] / df["Kplus_ETA_inv"]

        self.df = df
        return self.df

    def feature_engineering(self, all_df):
        """
        Features by david 05/11
        """

        all_df.columns = [col.replace(" ", "") for col in all_df.columns]
        # cos -> sin transformation
        all_df["Kst_892_0_sinThetaH"] = np.sqrt(1 - all_df["Kst_892_0_cosThetaH"] ** 2)
        all_df["B_DIRA_OWNPV_sin"] = np.sqrt(1 - all_df["B_DIRA_OWNPV"] ** 2)
        # x and y P components
        all_df["Kplus_P_x"] = all_df["Kplus_P"] * all_df["Kst_892_0_sinThetaH"]
        all_df["Kplus_P_y"] = all_df["Kplus_P"] * all_df["Kst_892_0_cosThetaH"]
        all_df["Kplus_P_x0"] = all_df["Kplus_P"] * np.sin(all_df["Kplus_ETA"])
        all_df["Kplus_P_y0"] = all_df["Kplus_P"] * np.cos(all_df["Kplus_ETA"])
        all_df["pminus_P_x"] = all_df["piminus_P"] * all_df["Kst_892_0_sinThetaH"]
        all_df["pminus_P_y"] = all_df["piminus_P"] * all_df["Kst_892_0_cosThetaH"]
        all_df["pminus_P_x0"] = all_df["piminus_P"] * np.sin(all_df["piminus_ETA"])
        all_df["pminus_P_y0"] = all_df["piminus_P"] * np.cos(all_df["piminus_ETA"])
        all_df["B_PT_x"] = all_df["B_PT"] * all_df["B_DIRA_OWNPV"]
        all_df["B_PT_y"] = all_df["B_PT"] * all_df["B_DIRA_OWNPV_sin"]
        # Full p
        # all_df["kp_x0_total"] = all_df["Kplus_P_x0"] + all_df["pminus_P_x0"] + all_df["B_PT_x"]
        # all_df["kp_x_total"] = all_df["Kplus_P_x"] + all_df["pminus_P_x"] + all_df["B_PT_x"]
        # all_df["kp_y0_total"] = all_df["Kplus_P_y0"] + all_df["pminus_P_y0"] + all_df["B_PT_y"]
        # all_df["kp_y_total"] = all_df["Kplus_P_y"] + all_df["pminus_P_y"] + all_df["B_PT_y"]
        # Paired p
        all_df["kp_x0"] = all_df["Kplus_P_x0"] + all_df["pminus_P_x0"]
        all_df["kp_x"] = all_df["Kplus_P_x"] + all_df["pminus_P_x"]
        all_df["kp_y0"] = all_df["Kplus_P_y0"] + all_df["pminus_P_y0"]
        all_df["kp_y"] = all_df["Kplus_P_y"] + all_df["pminus_P_y"]
        all_df["kbx0"] = all_df["Kplus_P_x0"] + all_df["B_PT_x"]
        all_df["kb_x"] = all_df["Kplus_P_x"] + all_df["B_PT_x"]
        all_df["kby0"] = all_df["Kplus_P_y0"] + all_df["B_PT_y"]
        all_df["kb_y"] = all_df["Kplus_P_y"] + all_df["B_PT_y"]
        all_df["kp_x0_minus"] = all_df["Kplus_P_x0"] - all_df["pminus_P_x0"]
        all_df["kp_x_minus"] = all_df["Kplus_P_x"] - all_df["pminus_P_x"]
        all_df["kp_y0_minus"] = all_df["Kplus_P_y0"] - all_df["pminus_P_y0"]
        all_df["kp_y_minus"] = all_df["Kplus_P_y"] - all_df["pminus_P_y"]
        all_df["kbx0_minus"] = all_df["Kplus_P_x0"] - all_df["B_PT_x"]
        all_df["kb_x_minus"] = all_df["Kplus_P_x"] - all_df["B_PT_x"]
        all_df["kby0_minus"] = all_df["Kplus_P_y0"] - all_df["B_PT_y"]
        all_df["kb_y_minus"] = all_df["Kplus_P_y"] - all_df["B_PT_y"]
        all_df["kp_abs_x0"] = np.abs(all_df["Kplus_P_x0"]) + np.abs(
            all_df["pminus_P_x0"]
        )
        all_df["kp_abs_x"] = np.abs(all_df["Kplus_P_x"]) + np.abs(all_df["pminus_P_x"])
        all_df["kp_abs_y0"] = np.abs(all_df["Kplus_P_y0"]) + np.abs(
            all_df["pminus_P_y0"]
        )
        all_df["kp_abs_y"] = np.abs(all_df["Kplus_P_y"]) + np.abs(all_df["pminus_P_y"])
        # Full p ratio
        all_df["kp_x0_ratio"] = (all_df["Kplus_P_x0"] + all_df["pminus_P_x0"]) / all_df[
            "Kplus_P"
        ]
        all_df["kp_x_ratio"] = (all_df["Kplus_P_x"] + all_df["pminus_P_x"]) / all_df[
            "Kplus_P"
        ]
        all_df["kp_y0_ratio"] = (all_df["Kplus_P_y0"] + all_df["pminus_P_y0"]) / all_df[
            "Kplus_P"
        ]
        all_df["kp_y_ratio"] = (all_df["Kplus_P_y"] + all_df["pminus_P_y"]) / all_df[
            "Kplus_P"
        ]
        all_df["kbx0_ratio"] = (all_df["Kplus_P_x0"] + all_df["B_PT_x"]) / all_df[
            "Kplus_P"
        ]
        all_df["kb_x_ratio"] = (all_df["Kplus_P_x"] + all_df["B_PT_x"]) / all_df[
            "Kplus_P"
        ]
        all_df["kby0_ratio"] = (all_df["Kplus_P_y0"] + all_df["B_PT_y"]) / all_df[
            "Kplus_P"
        ]
        all_df["kb_y_ratio"] = (all_df["Kplus_P_y"] + all_df["B_PT_y"]) / all_df[
            "Kplus_P"
        ]
        all_df["kp_x0_minus_ratio"] = (
            all_df["Kplus_P_x0"] - all_df["pminus_P_x0"]
        ) / all_df["Kplus_P"]
        all_df["kp_x_minus_ratio"] = (
            all_df["Kplus_P_x"] - all_df["pminus_P_x"]
        ) / all_df["Kplus_P"]
        all_df["kp_y0_minus_ratio"] = (
            all_df["Kplus_P_y0"] - all_df["pminus_P_y0"]
        ) / all_df["Kplus_P"]
        all_df["kp_y_minus_ratio"] = (
            all_df["Kplus_P_y"] - all_df["pminus_P_y"]
        ) / all_df["Kplus_P"]
        all_df["kbx0_minus_ratio"] = (all_df["Kplus_P_x0"] - all_df["B_PT_x"]) / all_df[
            "Kplus_P"
        ]
        all_df["kb_x_minus_ratio"] = (all_df["Kplus_P_x"] - all_df["B_PT_x"]) / all_df[
            "Kplus_P"
        ]
        all_df["kby0_minus_ratio"] = (all_df["Kplus_P_y0"] - all_df["B_PT_y"]) / all_df[
            "Kplus_P"
        ]
        all_df["kb_y_minus_ratio"] = (all_df["Kplus_P_y"] - all_df["B_PT_y"]) / all_df[
            "Kplus_P"
        ]
        # # things in hbar units
        # all_df["B_hbar"] = all_df["B_PT"] * all_df["B_IPCHI2_OWNPV"]
        # all_df["B_hbar_2"] = all_df["B_PT"] * all_df["B_FDCHI2_OWNPV"]
        # all_df["K_hbar"] = all_df["Kplus_P"] * all_df["Kplus_IP_OWNPV"]
        # all_df["p_hbar"] = all_df["piminus_P"] * all_df["piminus_IP_OWNPV"]
        #
        # # hbar ratios
        # all_df["B_hbar_ratio"] = all_df["B_hbar"] / all_df["B_hbar_2"]
        # all_df["K_p_hbar_ratio"] = all_df["K_hbar"] / all_df["p_hbar"]
        # all_df["K_B_hbar_ratio"] = all_df["K_hbar"] / all_df["B_hbar"]
        # p ratios
        all_df["gamma_B_PT_ratio"] = all_df["gamma_PT"] / all_df["B_PT"]
        all_df["piminus_B_P_ratio"] = all_df["piminus_P"] / all_df["B_PT"]
        # all_df["piminus_B_P_ratio_x"] = (all_df["pminus_P_x"] / all_df['B_PT'])
        # all_df["piminus_B_P_ratio_y"] = (all_df["pminus_P_y"] / all_df['B_PT'])
        all_df["kplus_B_P_ratio"] = all_df["Kplus_P"] / all_df["B_PT"]
        # all_df["kplus_B_P_ratio_x"] = (all_df["Kplus_P_x"] / all_df['B_PT'])
        # all_df["kplus_B_P_ratio_y"] = (all_df["Kplus_P_y"] / all_df['B_PT'])
        all_df["kplus_piminus_P_ratio"] = all_df["Kplus_P"] / all_df["piminus_P"]
        # distance ratios
        all_df["b_distance_ratio"] = all_df["B_IPCHI2_OWNPV"] / all_df["B_FDCHI2_OWNPV"]
        all_df["k_p_distance_ratio"] = (
            all_df["Kplus_IP_OWNPV"] / all_df["piminus_IP_OWNPV"]
        )
        all_df["k_b_distance_ratio"] = (
            all_df["Kplus_IP_OWNPV"] / all_df["B_IPCHI2_OWNPV"]
        )
        all_df["p_b_distance_ratio"] = (
            all_df["piminus_IP_OWNPV"] / all_df["B_IPCHI2_OWNPV"]
        )
        all_df["k_kst_distance_ratio"] = (
            all_df["Kplus_IP_OWNPV"] / all_df["Kst_892_0_IP_OWNPV"]
        )
        # shpere radius
        # all_df["sphere_radius_k_b"] =  all_df['Kplus_IP_OWNPV']**2 + all_df['B_IPCHI2_OWNPV']**2
        # all_df["sphere_radius_p_b"] =  all_df['piminus_IP_OWNPV']**2 + all_df['B_IPCHI2_OWNPV']**2
        # ANGLE ratios
        # all_df["B_DIR"] = np.arccos(all_df["B_DIRA_OWNPV"])
        # all_df["theta"] = np.arccos(all_df["Kst_892_0_cosThetaH"])
        # all_df["angle_ratio"] = np.log(all_df["B_DIR"] / all_df["theta"])
        # all_df["b_eta"] = np.arccos(all_df["B_DIRA_OWNPV"])
        # all_df["b_K_ratio"] = all_df["b_eta"] / all_df["Kplus_ETA"]
        # ETA ratio
        all_df["eta_ratio"] = all_df["Kplus_ETA"] / all_df["piminus_ETA"]
        # Conservation of momentum
        all_df["total_momentum"] = (
            all_df["gamma_PT"]
            + all_df["Kplus_P"]
            + all_df["piminus_P"]
            - all_df["B_PT"]
        )
        all_df["total_momentum1"] = (
            all_df["gamma_PT"]
            - all_df["Kplus_P"]
            + all_df["piminus_P"]
            - all_df["B_PT"]
        )
        all_df["total_momentum2"] = (
            all_df["gamma_PT"]
            + all_df["Kplus_P"]
            - all_df["piminus_P"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_sq"] = (
            all_df["gamma_PT"] ** 2
            + all_df["Kplus_P"] ** 2
            + all_df["piminus_P"] ** 2
            - all_df["B_PT"] ** 2
        )
        all_df["total_momentum_x"] = (
            all_df["gamma_PT"]
            + all_df["Kplus_P_x"]
            + all_df["pminus_P_x"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_x2p"] = (
            all_df["gamma_PT"]
            + all_df["Kplus_P_x"]
            + 2 * all_df["pminus_P_x"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_x0"] = (
            all_df["gamma_PT"]
            + all_df["Kplus_P_x"]
            + all_df["pminus_P_x"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_x1"] = (
            all_df["gamma_PT"]
            - all_df["Kplus_P_x"]
            + all_df["pminus_P_x"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_x2"] = (
            all_df["gamma_PT"]
            + all_df["Kplus_P_x"]
            - all_df["pminus_P_x"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_y"] = (
            all_df["gamma_PT"]
            + all_df["Kplus_P_y"]
            + all_df["pminus_P_y"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_y2p"] = (
            all_df["gamma_PT"]
            + all_df["Kplus_P_y"]
            + 2 * all_df["pminus_P_y"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_y1"] = (
            all_df["gamma_PT"]
            - all_df["Kplus_P_y"]
            + all_df["pminus_P_y"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_y2"] = (
            all_df["gamma_PT"]
            + all_df["Kplus_P_y"]
            - all_df["pminus_P_y"]
            - all_df["B_PT"]
        )
        all_df["total_momentum_abs"] = np.abs(all_df["total_momentum"])
        all_df["total_momentum_x_abs"] = np.abs(all_df["total_momentum_x"])
        all_df["total_momentum_y_abs"] = np.abs(all_df["total_momentum_y"])
        # all_df["b_pi_dff_P_y"] = 2 * all_df["pminus_P_y"] - all_df["B_PT"]
        # all_df["b_pi_dff_P_x"] = 2 * all_df["pminus_P_x"] - all_df["B_PT"]
        all_df["total_momentum_K"] = (
            all_df["gamma_PT"] + all_df["Kplus_P"] - all_df["B_PT"]
        )
        all_df["total_momentum_K_x"] = (
            all_df["gamma_PT"] + all_df["Kplus_P_x"] - all_df["B_PT_x"]
        )
        all_df["total_momentum_K_y"] = (
            all_df["gamma_PT"] + all_df["Kplus_P_y"] - all_df["B_PT_y"]
        )
        all_df["total_momentum_K_abs"] = np.abs(all_df["total_momentum_K"])
        all_df["total_momentum_K_x_abs"] = np.abs(all_df["total_momentum_K_x"])
        all_df["total_momentum_K_y_abs"] = np.abs(all_df["total_momentum_K_y"])
        all_df["total_momentum_p"] = (
            all_df["gamma_PT"] + all_df["piminus_P"] - all_df["B_PT"]
        )
        all_df["total_momentum_p_x"] = (
            all_df["gamma_PT"] + all_df["pminus_P_x"] - all_df["B_PT_x"]
        )
        all_df["total_momentum_p_y"] = (
            all_df["gamma_PT"] + all_df["pminus_P_y"] - all_df["B_PT_y"]
        )
        all_df["total_momentum_p_abs"] = np.abs(all_df["total_momentum_p"])
        all_df["total_momentum_p_x_abs"] = np.abs(all_df["total_momentum_p_x"])
        all_df["total_momentum_p_y_abs"] = np.abs(all_df["total_momentum_p_y"])

        return all_df

    def __len__(self):
        return len(self.X)

    def __shape__(self):
        return self.X.shape[1]

    def __getitem__(self, idx):
        # Convert idx from tensor to list due to pandas bug (that arises when using pytorch's random_split)
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        return [self.X.iloc[idx].values, self.y[idx]]


class Net(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 6 * input_dim)
        self.relu1 = nn.SELU()
        self.batchnorm1 = nn.BatchNorm1d(6 * input_dim)
        self.drop1 = nn.Dropout(0.05, inplace=False)

        self.fc2 = nn.Linear(6 * input_dim, 3 * input_dim, bias=False)
        self.relu2 = nn.SELU()
        self.batchnorm2 = nn.BatchNorm1d(
            3 * input_dim,
            eps=1e-05,
            momentum=0.1,
            affine=True,
            track_running_stats=True,
        )
        self.drop2 = nn.Dropout(0.05, inplace=False)

        self.fc3 = nn.Linear(3 * input_dim, 2 * input_dim, bias=False)
        self.relu3 = nn.SELU()
        self.batchnorm3 = nn.BatchNorm1d(
            2 * input_dim,
            eps=1e-05,
            momentum=0.1,
            affine=True,
            track_running_stats=True,
        )
        self.drop3 = nn.Dropout(0.05, inplace=False)

        self.fc4 = nn.Linear(2 * input_dim, 1 * input_dim, bias=False)
        self.relu4 = nn.SELU()
        self.batchnorm4 = nn.BatchNorm1d(
            input_dim, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        )
        self.drop4 = nn.Dropout(0.05, inplace=False)

        self.fc5 = nn.Linear(input_dim, 1, bias=True)

        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        self.batchnorm1(x)
        self.drop1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        self.batchnorm2(x)
        self.drop2(x)

        x = self.fc3(x)
        x = self.relu3(x)
        self.batchnorm3(x)
        self.drop3(x)

        x = self.fc4(x)
        x = self.relu4(x)
        self.batchnorm4(x)
        self.drop4(x)

        x = self.fc5(x)
        x = self.sig(x)

        return x.squeeze()

    def step(self, inputs):
        data, label = inputs  # ignore label
        outputs = self.forward(data)
        _, preds = torch.max(outputs.data, 1)
        # preds, outputs  are cuda tensors. Right?
        return preds, outputs

    def predict_proba(self, data):
        # Pass the NN forward
        out = self.forward(data)

        # Transform to
        return out.detach().numpy().squeeze()


class ResNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 6 * input_dim)
        self.relu1 = nn.SELU()
        self.batchnorm1 = nn.BatchNorm1d(6 * input_dim)
        self.drop1 = nn.Dropout(0.05, inplace=False)

        self.fc2 = nn.Linear(6 * input_dim, 3 * input_dim, bias=False)
        self.relu2 = nn.SELU()
        self.batchnorm2 = nn.BatchNorm1d(
            3 * input_dim ,
            eps=1e-05,
            momentum=0.1,
            affine=True,
            track_running_stats=True,
        )
        self.drop2 = nn.Dropout(0.05, inplace=False)

        self.fc3 = nn.Linear(3 * input_dim, 2 * input_dim, bias=False)
        self.relu3 = nn.SELU()
        self.batchnorm3 = nn.BatchNorm1d(
            2 * input_dim+ input_dim,
            eps=1e-05,
            momentum=0.1,
            affine=True,
            track_running_stats=True,
        )
        self.drop3 = nn.Dropout(0.05, inplace=False)

        self.fc4 = nn.Linear(2 * input_dim+ input_dim, 1 * input_dim, bias=False)
        self.relu4 = nn.SELU()
        self.batchnorm4 = nn.BatchNorm1d(
            input_dim, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        )
        self.drop4 = nn.Dropout(0.05, inplace=False)

        self.fc5 = nn.Linear(input_dim, 1, bias=True)

        self.sig = nn.Sigmoid()

    def forward(self, x):
        x1 = x
        x = self.fc1(x)
        x = self.relu1(x)
        self.batchnorm1(x)
        self.drop1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        self.batchnorm2(x)
        self.drop2(x)

        x = self.fc3(x)
        x = self.relu3(torch.cat((x, x1), 1))
        self.batchnorm3(x)
        self.drop3(x)

        x = self.fc4(x)
        x = self.relu4(x)
        self.batchnorm4(x)
        self.drop4(x)

        x = self.fc5(x)
        x = self.sig(x)

        return x.squeeze()

    def step(self, inputs):
        data, label = inputs  # ignore label
        outputs = self.forward(data)
        _, preds = torch.max(outputs.data, 1)
        # preds, outputs  are cuda tensors. Right?
        return preds, outputs

    def predict_proba(self, data):
        # Pass the NN forward
        out = self.forward(data)

        # Transform to
        return out.detach().numpy().squeeze()


class BasicBlock(nn.Module):
    def __init__(self, siz3, downsample=None):
        super().__init__()
        self.fc1 = nn.Linear(size, 2 * size)

        self.bn1 = nn.BatchNorm1d(2 * size)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(2 * size, size)
        self.bn2 = nn.BatchNorm1d(size)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.fc2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

    def _make_layer(block, inplanes, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(inplanes, planes, stride, downsample))
        inplanes = planes
        for _ in range(1, blocks):
            layers.append(block(inplanes, planes))
        return nn.Sequential(*layers)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [7]:
# coding: utf-8


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn")
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

import time

from sklearn.metrics import roc_auc_score


def evaluate_auc(model, data, label):
    return roc_auc_score(label.detach().numpy(), model(data.float()).detach().numpy())


tic = time.time()

# Read data
train_file = "train_split.csv"
validation_file = "valid_split.csv"
trainset = ReadDataset(train_file)
testset = ReadDataset(validation_file)

# Data loaders
trainloader = DataLoader(trainset, batch_size=100, shuffle=True)

# Test set
X_test = torch.tensor(testset.X.values)
y_test = torch.tensor(testset.y.values)
print(trainset.X.shape)
print(testset.X.shape)



# Use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Neural Network
nnet = ResNet(trainset.__shape__()).to(device)

# Loss function
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(
    nnet.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.000001
)


# Train the net
loss_per_iter = []
loss_per_batch = []


# Train the net
losses = []
auc_train = []
auc_test = []

# hyperparameteres
n_epochs = 4


---
(159496, 182)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


(159496, 582)
---
(53166, 182)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


(53166, 580)
(159496, 582)
(53166, 580)


In [6]:
#!/usr/bin/env python

for epoch in range(n_epochs):
    print(epoch)

    for i, (inputs, labels) in enumerate(trainloader):
        X = inputs.to(device)
        y = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forwarde
        outputs = nnet(X.float())

        # Compute diff
        loss = criterion(outputs, y.float())

        # Compute gradient
        loss.backward()

        # update weights
        optimizer.step()

        # Save loss to plot

        losses.append(loss.item())

        if i % 500 == 0:
            auc_train.append(evaluate_auc(nnet, X.float(), y.float()))
            auc_test.append(evaluate_auc(nnet, X_test, y_test))

            # Figure
            plt.figure()
            plt.ylim([0.6, 1])
            plt.plot(auc_test, label="test")
            plt.plot(auc_train, label="train")
            plt.legend()
            plt.savefig("output/auc_NN.png")
            plt.savefig("output/auc_NN.svg", format="svg")
            plt.close()

    if epoch % 10 == 0:
        path = "output/weights" + str(epoch) + ".pt"
        torch.save(nnet.state_dict(), path)

torch.save(nnet.state_dict(), "output/weights_final.pt")
toc = time.time()
elap_time = np.round(np.abs(tic - toc), 1)
print("Elapsed time: ", elap_time)
print("done")


---
(159496, 182)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


(159496, 582)
---
(53166, 182)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


(53166, 580)
(159496, 582)
(53166, 580)
0


RuntimeError: size mismatch, m1: [53166 x 580], m2: [582 x 3492] at /Users/distiller/project/conda/conda-bld/pytorch_1595629430416/work/aten/src/TH/generic/THTensorMath.cpp:41

In [9]:
gb = GradientBoostingFeatureGenerator()

In [22]:
gb.fit(trainset.X.head(10),trainset.y.head(10))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


GradientBoostingFeatureGenerator(add_probs=False, criterion='friedman_mse',
                                 init=None, learning_rate=0.1, loss='deviance',
                                 max_depth=3, max_features=None,
                                 max_leaf_nodes=None, min_impurity_decrease=0.0,
                                 min_impurity_split=None, min_samples_leaf=1,
                                 min_samples_split=2,
                                 min_weight_fraction_leaf=0.0, n_estimators=50,
                                 n_iter_no_change=None, presort='auto',
                                 random_state=None, sparse_feat=False,
                                 stack_to_X=True, subsample=1.0, tol=0.0001,
                                 validation_fraction=0.1, verbose=0,
                                 warm_start=False)

In [23]:
gb.transform(trainset.X.head(100)).shape

(100, 682)

In [32]:
gb.transform(trainset.X.tail(10)).shape

(10, 682)